<a href="https://colab.research.google.com/github/fangzhongfionaxu/DeepLearning04/blob/main/Deep_Learning_assignment_4_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

# metadata
print(covertype.metadata)

# # variable information
# print(covertype.variables)

{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [3]:
#get missing columns
missing_cols = X.columns[X.isnull().any()]
print(missing_cols)
# no missing columns

Index([], dtype='object')


In [4]:
# scaling
# Vertical_Distance_To_Hydrology standard scaler
# others min max scaler

from sklearn.preprocessing import StandardScaler, MinMaxScaler
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
print(numerical_columns)

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Soil_Type1',
       'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
       'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4

In [5]:
standard_cols = ['Elevation', 'Vertical_Distance_To_Hydrology']
binary_cols = [col for col in X.columns if set(X[col].unique()) <= {0, 1}]
print(binary_cols)

['Wilderness_Area1', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4']


In [6]:
minmax_cols = [col for col in numerical_columns if col not in standard_cols + binary_cols]
print(minmax_cols)

['Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']


In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_standard_scaled = StandardScaler().fit_transform(X[standard_cols])
X_minmax_scaled = MinMaxScaler().fit_transform(X[minmax_cols])
X_scaled = pd.DataFrame(index=X.index)

# scaled
X_scaled[standard_cols] = pd.DataFrame(X_standard_scaled, columns=standard_cols, index=X.index)
X_scaled[minmax_cols] = pd.DataFrame(X_minmax_scaled, columns=minmax_cols, index=X.index)

# untouched binary features
X_scaled[binary_cols] = X[binary_cols]
X_scaled = X_scaled[X.columns] # keep original column order

In [8]:
X_scaled.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4
0,-1.297805,0.141667,0.045455,0.184681,-0.796273,0.071659,0.870079,0.913386,0.582677,0.875366,...,0,0,0,0,0,0,0,0,0,0
1,-1.319235,0.155556,0.030303,0.151754,-0.899197,0.054798,0.866142,0.925197,0.594488,0.867838,...,0,0,0,0,0,0,0,0,0,0
2,-0.554907,0.386111,0.136364,0.191840,0.318742,0.446817,0.921260,0.937008,0.531496,0.853339,...,0,0,0,0,0,0,0,0,0,0
3,-0.622768,0.430556,0.272727,0.173228,1.227908,0.434172,0.937008,0.937008,0.480315,0.865886,...,0,0,0,0,0,0,0,0,0,0
4,-1.301377,0.125000,0.030303,0.109520,-0.813427,0.054939,0.866142,0.921260,0.590551,0.860449,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# check if there are categorical var
categorical_cols = X.select_dtypes(include=['object']).columns
print(categorical_cols)

Index([], dtype='object')


In [10]:
# split into train and val
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [11]:
X_train.shape

(464809, 54)

Build model

In [12]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, ReLU, Add, Input
from tensorflow.keras.optimizers import Adam

input_features = X_train.shape[1]
inputs = Input(shape=(input_features,))
inputs.shape

(None, 54)

In [13]:
def residual_block(x):

    y = Dense(256, activation='relu')(x)
    y = Dense(256, activation='relu')(y)
    residual = Add()([x, y])
    return residual

In [27]:

dense_layer1 = Dense(256, activation='relu')(inputs)
dense_layer2 = Dense(256, activation='relu')(dense_layer1)
res_connection = Add()([dense_layer2, dense_layer1])

skip = Dense(256)(inputs)
merged = Add()([res_connection, skip])

dense_layer3 = Dense(256, activation='relu')(merged)
outputs = Dense(7, activation='softmax')(dense_layer3)
model = Model(inputs=inputs, outputs=outputs)

In [15]:
model.save("residual_model.h5")

from google.colab import files
files.download('/content/residual_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
print(os.getcwd())

/content


Overfit

In [30]:
X_batch = X_train[:128]
y_batch = y_train[:128]
print(X_batch.shape)
print(y_batch.shape)

(128, 54)
(128, 1)


In [31]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [35]:
print(np.unique(y_train))
num_classes = len(np.unique(y_train))
print(num_classes)
print(y_batch.shape)

[1 2 3 4 5 6 7]
7
(128, 1)


In [36]:
from tensorflow.keras.utils import to_categorical

# Assuming you have 7 classes, change as needed
num_classes = len(np.unique(y_train))

y_batch = to_categorical(y_batch-1, num_classes)
y_val = to_categorical(y_val-1, num_classes)

In [38]:
history = model.fit(
    X_batch, y_batch,
    epochs=200,            # Adjust as needed
    batch_size=128,        # Full batch training
    validation_data=(X_val, y_val),  # Optional: for contrast
    verbose=1

)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.8672 - loss: 0.3941 - val_accuracy: 0.6189 - val_loss: 0.9720
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8672 - loss: 0.3809 - val_accuracy: 0.6190 - val_loss: 0.9798
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.8672 - loss: 0.3683 - val_accuracy: 0.6194 - val_loss: 0.9890
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8594 - loss: 0.3566 - val_accuracy: 0.6196 - val_loss: 0.9994
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.8516 - loss: 0.3456 - val_accuracy: 0.6204 - val_loss: 1.0106
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.8516 - loss: 0.3350 - val_accuracy: 0.6208 - val_loss: 1.0234
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.8594 - loss: 0.3246 - val_accuracy: 0.6207 - val_loss: 1.0382
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.8750 - loss: 0.3145 - val_accuracy: 0.6197 - val_loss: 1.0548
Epoch 9/

In [40]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 256)            │         14,080 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 256)            │         65,792 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 256)            │              0 │ dense_6[0][0],         │
│                           │                        │                │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 256)            │         14,080 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 256)            │              0 │ add_2[0][0],           │
│                           │                        │                │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 256)            │         65,792 │ add_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 7)              │          1,799 │ dense_8[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 484,631 (1.85 MB)

 Trainable params: 161,543 (631.03 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 323,088 (1.23 MB)

In [41]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)

In [42]:
print(f"Number of parameters: {model.count_params()}")
print(f"Final training loss: {history.history['loss'][-1]}")
print(f"Final validation loss: {val_loss}")

Number of parameters: 161543
Final training loss: 0.0010781318414956331
Final validation loss: 4.085626125335693
